In [169]:
import pandas as pd
import pyreadr
import numpy as np
from tqdm import tqdm

In [72]:
path = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito.Rdata'
pathvote = '/Users/annieulichney/Library/CloudStorage/Box-Box/Deforestation_MachineLearning/Data/cand_prefeito_vote.Rdata'

df = pyreadr.read_r(path)['cand']
vote = pyreadr.read_r(pathvote)['cand']



In [73]:
#for each mun and each year want winning candidate, bio info, margin etc, 

#votes, total votes, pct votes, political party, incumbent runs, incumbent wins, seat limit

In [74]:
vote.head()

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989


In [210]:
keep = vote[vote.resultado == 'eleito'][['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido' ,'votos']]
keep.columns = ['ano', 'id_municipio', 'id_candidato_bd', 'sigla_partido', 'votos_winner']

#total votes for that municipio that year
keep = pd.merge(keep, vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index(), how = 'left', left_on = ['ano', 'id_municipio'], right_on = ['ano', 'id_municipio'])
keep.head()

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos
0,2000,2708907,1265591,PSD,3147,4882
1,2000,1302553,19147,PST,1455,5658
2,2000,2903300,1071793,PAN,2135,3598
3,2000,2906907,962907,PSD,4614,8760
4,2000,2908606,1523132,PST,3643,8872


In [211]:
keep['incumbant_run'] = np.zeros(keep.shape[0])
keep['incumbant_win'] = np.zeros(keep.shape[0])
keep['term_lim_seat'] = np.zeros(keep.shape[0])
keep['margin'] = np.zeros(keep.shape[0])

In [217]:
#first find does incumbant run and does incumbant win? 
missing_incumbant = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    row

    #did they win? 
    if row.resultado == 'eleito':
        #check 4 years from current year for year+4 and id match
        incumbant_run = vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].shape[0]
        
       
        #if they run in 4 years, check if they win 
        if incumbant_run == 1:
            if vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].shape[0] == 0:
                print('Missing data with :' + str(i))

            else:
                incumbant_win = int(vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)].resultado == 'eleito')
        
        
        #if they do not run in 4 years, win is 0
        if incumbant_run==0:
            incumbant_win = 0
        
        #stop if year is 2020 because 2024 not in dataset
        if row.ano == 2020:
            continue
        
        #update incumbent run and incumbent win for each row
        if keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].shape[0] == 0:
            #print('Missing row at :' + str(i) + 'year = ' + str(row.ano+4) + ' munic = ' + str(row.id_municipio))
            missing_incumbant.append([i, row.ano+4,row.id_municipio])
        else:
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_run'] = incumbant_run
            keep.loc[keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)].index[0], 'incumbant_win'] = incumbant_win




100%|██████████| 96838/96838 [10:50<00:00, 148.97it/s] 


In [218]:
#now do if the seat is term limited: 
missing = []

for i in tqdm(range(vote.shape[0])):
    row = vote.iloc[i]
    
    temp_frame = vote[(vote.id_candidato_bd == row.id_candidato_bd)&(vote.resultado == 'eleito')]
    temp_frame = temp_frame.sort_values('ano').reset_index()
    
    for index, row2 in temp_frame.iterrows():
        if temp_frame[temp_frame.ano == row2.ano+4].shape[0] != 0:
            
            if keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].shape[0] == 0:
                missing.append([i,row.ano,row.id_municipio])
                #missing.append('Missing at ' + str(i) + ' year = ' + str(row.ano) + str(' munic = ') + str(row.id_municipio))
                                 
            else:
                keep.loc[keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].index[0], 'term_lim_seat'] = 1


    
    

100%|██████████| 96838/96838 [22:33<00:00, 71.52it/s] 


In [213]:
#now get margin of victory


vote_missing = []
for i in tqdm(range(keep.shape[0])):
    row = keep.iloc[i]
    
    if vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].shape[0] == 0:
        vote_missing.append(i, row.ano, row.id_municipio)
    else:
        vote_sort = sorted(vote[(vote.ano == row.ano) & (vote.id_municipio == row.id_municipio)].votos, reverse = True)
        vote_sort.append(0)
        margin = vote_sort[0] - vote_sort[1]
        
    if keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].shape[0] != 0:
        keep.loc[keep[(keep.ano == row.ano) & (keep.id_municipio == row.id_municipio)].index[0], 'margin'] = margin
        
        
        

100%|██████████| 33756/33756 [10:49<00:00, 52.01it/s]


,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat,margin
0,2000,2708907,1265591,PSD,3147,4882,0.0,0.0,0.0,1412.0
1,2000,1302553,19147,PST,1455,5658,0.0,0.0,0.0,155.0
2,2000,2903300,1071793,PAN,2135,3598,0.0,0.0,0.0,708.0
3,2000,2906907,962907,PSD,4614,8760,0.0,0.0,0.0,640.0
4,2000,2908606,1523132,PST,3643,8872,0.0,0.0,0.0,1015.0


In [166]:
i = 54

row = vote.iloc[i]
temp_frame = vote[(vote.id_candidato_bd == row.id_candidato_bd)&(vote.resultado == 'eleito')]
temp_frame = temp_frame.sort_values('ano').reset_index()

for index, row2 in temp_frame.iterrows():
    
    if temp_frame[temp_frame.ano == row2.ano+4].shape[0] != 0:
        keep.loc[keep[(keep.ano == row2.ano + 8) & (keep.id_municipio == row2.id_municipio)].index[0], 'term_lim_seat'] == 1
        
        

1


In [108]:
keep[(keep.ano == 2016) & (keep.id_municipio == '3305604')]

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat
14474,2016,3305604,831054,MDB,7470,14560,0.0,0.0,0.0


In [96]:
incumbant_win

0

In [100]:
keep[(keep.id_municipio == row.id_municipio)]

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat
403,2012,3305604,831054,PRB,6732,12953,0.0,0.0,0.0
6541,2008,3305604,883297,PT,6488,12353,0.0,0.0,0.0
14474,2016,3305604,831054,MDB,7470,14560,0.0,0.0,0.0
22633,2004,3305604,1630143,PFL,4831,7345,0.0,0.0,0.0
24445,2000,3305604,1630143,PSB,6371,10554,0.0,0.0,0.0


In [90]:
keep[(keep.ano == row.ano+4)&(keep.id_municipio == row.id_municipio)]

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,votos,incumbant_run,incumbant_win,term_lim_seat


In [251]:
vote.iloc[41]

ano                                  2000
turno                                   1
tipo_eleicao            eleicao ordinaria
sigla_uf                               AM
id_municipio                      1302553
id_municipio_tse                    98396
cargo                            prefeito
sigla_partido                         PST
numero_candidato                       18
sequencial_candidato                  NaN
id_candidato_bd                     19147
resultado                          eleito
votos                                1455
Name: 41, dtype: object

In [255]:
vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
27788,2004,1,eleicao ordinaria,AM,1302553,98396,prefeito,PTB,14,67,19147,nao eleito,954


In [238]:
keep[keep.incumbant_run  == 1]

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,incumbant_run,incumbant_win
41,2000,1302553,19147,PST,1455,1.0,NaN
49,2000,2903300,1071793,PAN,2135,1.0,NaN
54,2000,2908606,1523132,PST,3643,1.0,NaN
58,2000,2915908,614309,PSD,1138,1.0,NaN
74,2000,2300804,1542346,PSD,1837,1.0,NaN
...,...,...,...,...,...,...,...
82065,2004,4104105,1114055,PSDB,1459,1.0,0.0
82132,2004,4122909,879034,PSDB,1860,1.0,1.0
83832,2016,4104709,1273956,PSDB,5545,1.0,0.0
86738,2020,1500701,85805,PSDB,6306,1.0,0.0


In [196]:
vote[(vote.ano == 1996) & (vote.id_candidato_bd == 19147)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [198]:
keep[(keep.id_candidato_bd == 19147)]

,ano,id_municipio,id_candidato_bd,sigla_partido,votos_winner,incumbant_run


In [42]:
#is numero the same as numero candidato? 

In [43]:
#for each municipality each year want: 
#winning candidate, age, gender, party
#percent of votes
#incumbent runs? 


#incumbent? 
#incumbent runs? 
#margin


In [103]:
df[df.id_candidato_bd.isna() == False]

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,data_nascimento,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca
27,2000,eleicao ordinaria,CE,2305407,14095,511,32622546300,006587690701,21,41,...,1968-03-16,33,masculino,ensino medio completo,solteiro(a),brasileira,CE,Ico,NaN,NaN
28,2000,eleicao ordinaria,MG,3140001,47996,2757,32722702649,050039430213,468,41,...,1957-11-05,43,masculino,ensino superior completo,casado(a),brasileira,MG,Mariana,NaN,NaN
29,2000,eleicao ordinaria,PE,2606002,24198,10046,33080771400,005200620884,1,16,...,1951-06-22,49,masculino,ensino superior completo,casado(a),brasileira,PE,Garanhuns,NaN,NaN
30,2000,eleicao ordinaria,GO,5208707,93734,10291,33093873134,020052371058,246,16,...,1961-02-09,40,feminino,ensino superior completo,divorciado(a),brasileira,RS,Porto Alegre,NaN,NaN
31,2000,eleicao ordinaria,AM,1301951,98370,11119,01361864249,004578482216,201,18,...,1951-08-30,49,masculino,ensino medio incompleto,casado(a),brasileira,AM,Ipixuna,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100302,2020,eleicao ordinaria,MT,5106802,91359,1760144,30827310900,018444551856,110001042327,90,...,1959-03-22,62,masculino,ensino medio completo,divorciado(a),brasileira,RS,Terra De Areia,plananorte@gmailcom,branca
100303,2020,eleicao ordinaria,PE,2613404,25674,1763982,31008089400,037956490833,170000760838,90,...,1962-12-03,58,masculino,ensino superior completo,casado(a),brasileira,AL,São Luís Do Quitunde,aroldopajau@hotmailcom,branca
100304,2020,eleicao ordinaria,AM,1301209,2259,1768332,31221130234,012015422232,40000686733,90,...,1971-04-20,49,masculino,ensino fundamental completo,solteiro(a),brasileira,PB,Juazeirinho,lucianodefarias01amldf@gmailcom,parda
100305,2020,eleicao ordinaria,RJ,3306107,59218,1784783,32093632772,067680040361,190000995787,90,...,1955-11-14,65,masculino,ensino superior completo,casado(a),brasileira,RJ,Valença,pimentadacunha@uolcombr,preta


In [44]:
df.head()

,ano,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,id_candidato_bd,cpf,titulo_eleitoral,sequencial,numero,...,data_nascimento,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nascimento,municipio_nascimento,email,raca
0,1996,eleicao ordinaria,PI,2211001,12190,NaN,NaN,NaN,NaN,16,...,1966-11-18,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,eleicao ordinaria,SE,2800308,31054,NaN,NaN,NaN,NaN,16,...,1956-05-23,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,eleicao ordinaria,RS,4314902,88013,NaN,NaN,NaN,NaN,56,...,1950-07-12,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,eleicao ordinaria,MG,3106200,41238,NaN,NaN,NaN,NaN,16,...,1953-06-07,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,eleicao ordinaria,PB,2507507,20516,NaN,NaN,NaN,NaN,16,...,1953-04-04,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
df_keep = df[['ano', 'id_municipio', 'idade', 'genero', 'id_candidato_bd', 'ocupacao']]

vote_keep = vote[vote.resultado == 'eleito'][['ano', 'id_municipio', 'sigla_partido', 'id_candidato_bd']]

In [109]:
new = pd.merge(df_keep, vote_keep, how = 'left', left_on = ['ano', 'id_municipio','id_candidato_bd'], right_on = ['ano', 'id_municipio', 'id_candidato_bd'])

new['incumbent_runs'] = np.zeros(new.shape[0])
new['incumbent_wins'] = np.zeros(new.shape[0])
new['margin'] = np.zeros(new.shape[0])

In [120]:
new.head()

,ano,id_municipio,idade,genero,id_candidato_bd,ocupacao,sigla_partido
0,1996,2211001,30,NaN,NaN,NaN,PSDB
1,1996,2800308,40,NaN,NaN,NaN,PMDB
2,1996,4314902,46,NaN,NaN,NaN,PT
3,1996,3106200,43,NaN,NaN,NaN,NaN
4,1996,2507507,43,NaN,NaN,NaN,PMDB


In [138]:
new[(new.ano == row.ano + 4) & (new.id_candidato_bd == row.id_candidato_bd)].shape[0]

0

In [137]:
vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos


In [ ]:
#Is it possible that they run in a different municipality

In [139]:
#for each candidate
i = 5000
row = vote.iloc[i]

#if they won, mark a 1 if they run again in 4 years
#if row.resultado == 'eleito':
    

In [142]:
new[new.id_candidato_bd == row.id_candidato_bd]

,ano,id_municipio,idade,genero,id_candidato_bd,ocupacao,sigla_partido
5916,2008,2914208,52,masculino,1320326,agricultor,PP
7173,2004,2914208,48,masculino,1320326,agricultor,NaN
9699,2012,2914208,56,masculino,1320326,agricultor,PP
11203,2020,2914208,64,masculino,1320326,agricultor,PP
61544,2000,2914208,44,masculino,1320326,proprietario de estabelecimento comercial,NaN


In [141]:
row

ano                                  2008
turno                                   1
tipo_eleicao            eleicao ordinaria
sigla_uf                               BA
id_municipio                      2914208
id_municipio_tse                    35858
cargo                            prefeito
sigla_partido                          PP
numero_candidato                       11
sequencial_candidato                 3103
id_candidato_bd                   1320326
resultado                          eleito
votos                                2084
Name: 5000, dtype: object

In [140]:
vote[(vote.ano == row.ano + 4) & (vote.id_candidato_bd == row.id_candidato_bd)]

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
8662,2012,1,eleicao ordinaria,BA,2914208,35858,prefeito,PP,11,50000021992,1320326,eleito,2517


In [136]:
row

ano                                  2016
turno                                   1
tipo_eleicao            eleicao ordinaria
sigla_uf                               PE
id_municipio                      2605202
id_municipio_tse                    24031
cargo                            prefeito
sigla_partido                         PPL
numero_candidato                       54
sequencial_candidato         170000019274
id_candidato_bd                    228144
resultado                      nao eleito
votos                                 483
Name: 1000, dtype: object

In [112]:
new.iloc[0]

ano                   1996
id_municipio       2211001
idade                   30
genero                 NaN
id_candidato_bd        NaN
ocupacao               NaN
sigla_partido         PSDB
Name: 0, dtype: object

In [ ]:
#for each person that ran and won, check if they ran again in 4 years. if they ran again put a 1 in incumbent ran

row = 

In [111]:
vote

,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
0,1996,1,eleicao ordinaria,AL,2704302,27855,prefeito,PSD,41,NaN,NaN,nao eleito,2388
1,1996,1,eleicao ordinaria,AM,1302603,2550,prefeito,PSTU,16,NaN,NaN,nao eleito,1713
2,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PAN,26,NaN,NaN,nao eleito,15721
3,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PCO,29,NaN,NaN,nao eleito,705
4,1996,1,eleicao ordinaria,BA,2927408,38490,prefeito,PGT,30,NaN,NaN,nao eleito,989
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96833,2020,1,eleicao ordinaria,TO,1712009,73407,prefeito,PROS,90,270001209647,176118,nao eleito,40
96834,2020,1,eleicao ordinaria,TO,1718501,73229,prefeito,PROS,90,270000802777,375744,nao eleito,15
96835,2020,1,eleicao ordinaria,TO,1721000,73440,prefeito,PROS,90,270000767544,536638,nao eleito,18523
96836,2020,2,eleicao ordinaria,CE,2303709,13730,prefeito,PROS,90,60000718891,441082,eleito,83588


In [47]:
new = new[new.numero_candidato.isna() == False]
new.head()

,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato
27,2000,2305407,33,masculino,41,outros,PSD,41
34,2000,1400308,40,masculino,18,tecnico de contabilidade e de estatistica,PST,18
35,2000,1302553,31,masculino,18,medico,PST,18
44,2000,3136603,63,masculino,41,nao informada,PSD,41
52,2000,3157377,39,masculino,41,proprietario de estabelecimento comercial,PSD,41


In [48]:
#calculate percentage of votes of winning candidate

new = new.merge(vote[['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index())

new.columns = ['ano', 'id_municipio', 'idade', 'genero', 'numero', 'ocupacao',
       'sigla_partido', 'numero_candidato', 'votos_total']

new = new.merge(vote[vote.resultado=='eleito'][['ano', 'id_municipio', 'votos']].groupby(['ano', 'id_municipio']).agg('sum').reset_index())

new['pct_votos'] = (new.votos/new.votos_total*100)

In [49]:
new.head()


,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato,votos_total,votos,pct_votos
0,2000,2305407,33,masculino,41,outros,PSD,41,29867,15660,52.432451
1,2000,1400308,40,masculino,18,tecnico de contabilidade e de estatistica,PST,18,7902,4472,56.593268
2,2000,1400308,48,masculino,18,nao informada,PST,18,7902,4472,56.593268
3,2000,1302553,31,masculino,18,medico,PST,18,5658,1455,25.715801
4,2000,3136603,63,masculino,41,nao informada,PSD,41,3536,1838,51.979638


In [99]:
new['incumbent_runs'] = np.zeros(new.shape[0])
new['incumbent_wins'] = np.zeros(new.shape[0])
new['margin'] = np.zeros(new.shape[0])

In [ ]:
for i in range(new.shape[0]):
    

In [89]:
i = 10000

row = new.iloc[i]
row

#margin of victory
temp = vote[(vote.ano == row.ano)&(vote.id_municipio == row.id_municipio)]
margin = sorted(temp.votos.values, reverse = True)[0] - sorted(temp.votos.values, reverse = True)[1]

#check if winner ran and won 4 years ago, this means this winner won last time too so is incumbent
is_incumbent = vote[(vote.ano == row.ano-4)&(vote.id_municipio == row.id_municipio)&(vote.resultado == 'eleito')].shape[0]

#check if they win again twice? 

new.loc[i, 'margin'] = margin
new.loc[i, 'incumbent_wins'] = is_incumbent


In [102]:
vote[(vote.id_municipio == row.id_municipio)&(vote.resultado == 'eleito')]


,ano,turno,tipo_eleicao,sigla_uf,id_municipio,id_municipio_tse,cargo,sigla_partido,numero_candidato,sequencial_candidato,id_candidato_bd,resultado,votos
222,2000,1,eleicao ordinaria,MA,2107902,8575,prefeito,PSD,41,NaN,951640,eleito,3352
26893,2008,1,eleicao ordinaria,MA,2107902,8575,prefeito,PTB,14,10138,1747075,eleito,5659
27907,2004,1,eleicao ordinaria,MA,2107902,8575,prefeito,PTB,14,65,951640,eleito,5402
28934,2016,1,eleicao ordinaria,MA,2107902,8575,prefeito,PTB,14,100000002566,666578,eleito,8082
29740,2012,1,eleicao ordinaria,MA,2107902,8575,prefeito,PTB,14,100000015509,1747075,eleito,5966
30558,2020,1,eleicao ordinaria,MA,2107902,8575,prefeito,PTB,14,100001021231,666578,eleito,5330


In [ ]:
#for each person that 

vote[]

In [100]:
j = 1

this_row = vote.iloc[j]



ano                                  1996
turno                                   1
tipo_eleicao            eleicao ordinaria
sigla_uf                               AM
id_municipio                      1302603
id_municipio_tse                     2550
cargo                            prefeito
sigla_partido                        PSTU
numero_candidato                       16
sequencial_candidato                  NaN
id_candidato_bd                       NaN
resultado                      nao eleito
votos                                1713
Name: 1, dtype: object

In [ ]:
#if the winner wins again 

In [94]:
sorted(temp.votos.values, reverse = True)[0] - sorted(temp.votos.values, reverse = True)[1]

6183

In [90]:
row

ano                       2016
id_municipio           2107902
idade                       37
genero               masculino
numero                      14
ocupacao            empresario
sigla_partido              PTB
numero_candidato            14
votos_total               9981
votos                     8082
pct_votos             80.97385
Name: 10000, dtype: object

In [98]:
#did this person run 4 years ago and win? 
vote[(vote.ano == row.ano-4)&(vote.id_municipio == row.id_municipio)&(vote.resultado == 'eleito')].shape[0]

1

In [77]:
sorted(temp.votos.values, reverse = True)[0] - sorted(temp.votos.values, reverse = True)[1]

1453

In [24]:
inc = vote.copy()

inc['ano_4'] = inc.ano + 4

In [25]:
inc['incumbent_runs'] = np.zeros(inc.shape[0])

#fill in ones for winners, when you merge on 4 years from now 
#if the same id appears in 4 years, it will keep a 1 there 
inc.loc[inc.resultado == 'eleito', 'incumbent_runs'] = np.ones(inc[inc.resultado == 'eleito'].shape[0])

In [26]:
inc.loc[inc.resultado == 'eleito', 'incumbent_runs'] = np.ones(inc[inc.resultado == 'eleito'].shape[0])

In [27]:
(inc[['ano_4','id_municipio', 'incumbent_runs', 'numero_candidato']])

,ano_4,id_municipio,incumbent_runs,numero_candidato
0,2000,2704302,0.0,41
1,2000,1302603,0.0,16
2,2000,2927408,0.0,26
3,2000,2927408,0.0,29
4,2000,2927408,0.0,30
...,...,...,...,...
96833,2024,1712009,0.0,90
96834,2024,1718501,0.0,90
96835,2024,1721000,0.0,90
96836,2024,2303709,1.0,90


In [28]:
new = pd.merge(new, (inc[['ano_4','id_municipio', 'incumbent_runs', 'numero_candidato']]), how = 'left', left_on = ['ano', 'id_municipio','numero_candidato'], right_on = ['ano_4', 'id_municipio', 'numero_candidato'])




In [29]:
new.head()

,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato,votos_total,votos,pct_votos,ano_4,incumbent_runs
0,2000,2305407,33,masculino,41,outros,PSD,41,29867,15660,52.432451,NaN,NaN
1,2000,1400308,40,masculino,18,tecnico de contabilidade e de estatistica,PST,18,7902,4472,56.593268,NaN,NaN
2,2000,1400308,48,masculino,18,nao informada,PST,18,7902,4472,56.593268,NaN,NaN
3,2000,1302553,31,masculino,18,medico,PST,18,5658,1455,25.715801,NaN,NaN
4,2000,3136603,63,masculino,41,nao informada,PSD,41,3536,1838,51.979638,NaN,NaN


In [31]:
new['incumbent_runs'] = new['incumbent_runs'].fillna(0)

In [32]:
new.head()

,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato,votos_total,votos,pct_votos,ano_4,incumbent_runs
0,2000,2305407,33,masculino,41,outros,PSD,41,29867,15660,52.432451,NaN,0.0
1,2000,1400308,40,masculino,18,tecnico de contabilidade e de estatistica,PST,18,7902,4472,56.593268,NaN,0.0
2,2000,1400308,48,masculino,18,nao informada,PST,18,7902,4472,56.593268,NaN,0.0
3,2000,1302553,31,masculino,18,medico,PST,18,5658,1455,25.715801,NaN,0.0
4,2000,3136603,63,masculino,41,nao informada,PSD,41,3536,1838,51.979638,NaN,0.0


In [33]:
new[new.id_municipio == '4314175']

,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato,votos_total,votos,pct_votos,ano_4,incumbent_runs
157,2012,4314175,48,masculino,54,prefeito,PPL,54,3449,1840,53.348797,NaN,0.0
158,2012,4314175,48,masculino,45,empresario,PSDB,45,3449,1840,53.348797,2012.0,1.0
159,2012,4314175,43,masculino,45,produtor agropecuario,PSDB,45,3449,1840,53.348797,2012.0,1.0
4277,2020,4314175,55,masculino,11,prefeito,PP,11,1660,1298,78.192771,NaN,0.0
26400,2016,4314175,51,masculino,40,servidor publico estadual,PSB,40,1752,935,53.367580,NaN,0.0
27729,2000,4314175,36,masculino,45,nao informada,PSDB,45,1665,767,46.066066,NaN,0.0
28758,2008,4314175,44,masculino,45,medico,PSDB,45,1860,962,51.720430,2008.0,1.0
29552,2004,4314175,40,masculino,45,produtor agropecuario,PSDB,45,1704,1101,64.612676,2004.0,1.0


In [34]:
new[new.incumbent_runs==1]

,ano,id_municipio,idade,genero,numero,ocupacao,sigla_partido,numero_candidato,votos_total,votos,pct_votos,ano_4,incumbent_runs
158,2012,4314175,48,masculino,45,empresario,PSDB,45,3449,1840,53.348797,2012.0,1.0
159,2012,4314175,43,masculino,45,produtor agropecuario,PSDB,45,3449,1840,53.348797,2012.0,1.0
175,2020,2919504,57,masculino,18,agricultor,REDE,18,27153,14550,53.585239,2020.0,1.0
176,2020,2904308,37,masculino,18,prefeito,REDE,18,8384,3643,43.451813,2020.0,1.0
259,2016,3160306,52,masculino,10,prefeito,REPUBLICANOS,10,6868,2978,43.360513,2016.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34893,2020,1600212,49,masculino,90,prefeito,PROS,90,4066,1860,45.745204,2020.0,1.0
34907,2020,3129905,44,masculino,90,prefeito,PROS,90,2016,2016,100.000000,2020.0,1.0
34917,2020,2107803,70,masculino,90,prefeito,PROS,90,19121,10096,52.800586,2020.0,1.0
34919,2020,2402105,35,masculino,90,medico,PROS,90,6498,3314,51.000308,2020.0,1.0
